In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic_2d
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from plot_field import generate_afl_oval, plot_events
import seaborn as sns
import matplotlib.pyplot as plt

# Next position
chains_processed =  pd.read_pickle("../data/chains_processed.pkl")
# 1. Record scores against kicks

chains_processed['points'] = np.select([chains_processed.description == "Goal", chains_processed.description == "Behind"], [6, 1], default=np.NaN)
chains_processed['points'] = chains_processed['points'].shift(-1)

# 2. Reset goal positions
finCondList = [chains_processed.description == "Goal",
            chains_processed.behindInfo =='missLeft',
            chains_processed.behindInfo =='leftPost',
            chains_processed.behindInfo =='missRight',
            chains_processed.behindInfo =='touched',
            chains_processed.behindInfo =='rightPost']
## Y next pos
finychoiceList = [0,
              6.4,
              3.2,
              -6.4,
              0,
              -3.2]
## X next pos
finxchoiceList = [167/2] * 6

## Record condition for debugging
nextCondChoice = ['G',
                 'ML',
                 'LP',
                 'MR',
                 'Touched',
                 'RP']

chains_processed = chains_processed.assign(x = np.select(finCondList, finxchoiceList, default=chains_processed.x),
                                           y = np.select(finCondList, finychoiceList, default=chains_processed.y),
                                           Cond = np.select(finCondList, nextCondChoice, default='Def'))


chains_processed['x'] = np.where(chains_processed.description == "Goal", (167/2), chains_processed.x)
chains_processed['y'] = np.where(chains_processed.description == "Goal", 0, chains_processed.y)
# 2. shift next pos
chains_processed['x_next'] = chains_processed.groupby(['season', 'roundNumber', 'homeTeam', 'period'], as_index=False)['x'].shift(-1)
chains_processed['y_next'] = chains_processed.groupby(['season', 'roundNumber', 'homeTeam', 'period'], as_index=False)['y'].shift(-1)

finCondList = ((chains_processed.description == 'Spoil') & (~chains_processed.playingFor.shift(-1).isnull()) & \
    (~chains_processed.playingFor.shift(1).eq(chains_processed.playingFor.shift(-1)))) | \
    (chains_processed.possChng & chains_processed.finalPos & \
              (chains_processed.finalState.isin(["turnover", 'rushed'])) & \
              (chains_processed.description != 'Out On Full After Kick') & \
              (chains_processed.description.shift(-1) != 'Spoil'))

fCondTemp = [(chains_processed.description == 'Spoil') &  (~chains_processed.playingFor.shift(-1).isnull()) & \
    (~chains_processed.playingFor.shift(1).eq(chains_processed.playingFor.shift(-1))), 
    (chains_processed.possChng & chains_processed.finalPos & \
              (chains_processed.finalState.isin(["turnover", 'rushed'])) & \
              (~chains_processed.description.isin(['Out On Full', 'Out On Full After Kick'])) & \
              (chains_processed.description.shift(-1) != 'Spoil'))]

chains_processed = chains_processed.assign(x_next = np.where(finCondList, -chains_processed.x_next, chains_processed.x_next),
                                           y_next = np.where(finCondList, -chains_processed.y_next, chains_processed.y_next),
                                           Cond2 = np.select(fCondTemp, ['TO1', 'TO2'], 'Def'))
# 3. Remove game events
game_events = ['Out On Full After Kick', 'Out of Bounds', 'Ball Up Call', 'Goal', 'Behind','Kick Inside 50 Result', 'OOF Kick In']

chains_processed = chains_processed[~chains_processed.description.isin(game_events)]

# 4. Calculate distance for QA
def calc_dist(x0, y0, x1, y1):
    x0 = np.array(x0, dtype=float)
    y0 = np.array(y0, dtype=float)
    x1 = np.array(x1, dtype=float)
    y1 = np.array(y1, dtype=float)
    return np.sqrt((x1-x0)**2 + (y1 - y0)**2)

chains_processed = chains_processed.assign(possDist = calc_dist(chains_processed.x, chains_processed.y, chains_processed.x_next, chains_processed.y_next))


In [195]:
bin_period = 20


def bin_values(col, dir='x'):
    if dir == 'x':
        s = -1; e = 1
    else:
        s = -0.8; e = 0.8
    return pd.cut(col.values, pd.interval_range(start=s, end=e, periods=bin_period, closed='right'), include_lowest=True, precision=2)

x_bins = pd.interval_range(start=-1, end=1, periods=bin_period, closed='right')
y_bins = pd.interval_range(start=-0.8, end=0.8, periods=bin_period, closed='right')

chains_processed = chains_processed.assign(xInitialPoss_bin=bin_values(chains_processed.x, 'x'),
                                                yInitialPoss_bin=bin_values(chains_processed.y, 'y'),
                                                xFinalPoss_bin=bin_values(chains_processed.x_next, 'x'),
                                                yFinalPoss_bin=bin_values(chains_processed.y_next, 'y'))

disposal_actions = ['Kick', 'Handball', 'Spoil','Ground Kick', 'Kickin short', 'Kickin long', 'Kickin play on']

# movement_df = chains_processed[possession_summary.finalState.isin(disposal_actions) & ~(possession_summary.initialState == "Kickin play on")]

See here - https://soccermatics.readthedocs.io/en/latest/gallery/lesson4/plot_ExpectedThreat.html

## Step 1

Select only positive disposal type actions

TO DO: Include turnovers, out of bounds


In [ ]:
all_count = movement_df[['xInitialPoss_bin','yInitialPoss_bin']].value_counts()
move_count = movement_df[~movement_df.shotAtGoal][['xInitialPoss_bin','yInitialPoss_bin']].value_counts()
shot_count = movement_df[movement_df.shotAtGoal][['xInitialPoss_bin','yInitialPoss_bin']].value_counts()
score_count = movement_df[movement_df.goals > 0][['xInitialPoss_bin','yInitialPoss_bin']].value_counts() * 6  + movement_df[movement_df.behind > 0][['xInitialPoss_bin','yInitialPoss_bin']].value_counts()

move_probability = move_count.div(all_count, fill_value=0)
shot_probability = shot_count.div(all_count, fill_value=0)
score_probability = score_count.div(shot_count, fill_value=0).div(all_count.apply(lambda x: 1), fill_value=1)

In [ ]:
plot_df = movement_df[['xInitialPoss_bin','yInitialPoss_bin']].value_counts().reset_index()
plot_df['x'] = plot_df.xInitialPoss_bin.apply(lambda s: np.round(s.values.left,3))
plot_df['y'] = plot_df.yInitialPoss_bin.apply(lambda s: np.round(s.values.left,3))
# plot_df[['x', 'y']] = pd.DataFrame(plot_df['start_sector'].tolist(), index=plot_df.index)
plot_df = plot_df.pivot(index='y', columns='x', values='count')

fig=plt.figure()
ax=fig.add_subplot()
pcm = ax.imshow(plot_df, zorder=0, aspect="auto", extent=(-80,80, -70.5, 70.5), 
          cmap='Greens')
plt.colorbar(pcm)
generate_afl_oval('MCG')

In [ ]:
plot_df = movement_df[movement_df.shotAtGoal][['xInitialPoss_bin','yInitialPoss_bin']].value_counts().reset_index()
plot_df['x'] = plot_df.xInitialPoss_bin.apply(lambda s: np.round(s.values.left,3))
plot_df['y'] = plot_df.yInitialPoss_bin.apply(lambda s: np.round(s.values.left,3))
plot_df = plot_df.sort_values(by='y')
plot_df = plot_df.pivot(index='y', columns='x', values='count')
plot_df = plot_df.reindex(columns=sorted(plot_df.columns))

fig=plt.figure(figsize=(10, 6.5))
ax=fig.add_subplot()
pcm = ax.imshow(plot_df, zorder=0, aspect="auto", extent=(-80,80, -70.5, 70.5), 
          cmap='Greens')
plt.colorbar(pcm)
generate_afl_oval('MCG')

In [ ]:
plot_df = move_probability.reset_index()
plot_df['x'] = plot_df.xInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 80
plot_df['y'] = plot_df.yInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 70.5
plot_df = plot_df.sort_values(by='y')
plot_df = plot_df.pivot(index='y', columns='x', values='count')
plot_df = plot_df.reindex(columns=sorted(plot_df.columns))

fig=plt.figure(figsize=(10, 6.5))
ax=fig.add_subplot()
pcm = ax.imshow(plot_df, zorder=0, aspect="auto", extent=(-80,80, -70.5, 70.5), 
          cmap='Blues')
plt.colorbar(pcm)
generate_afl_oval('MCG')

In [ ]:
plot_df = shot_probability.reset_index()
plot_df['x'] = plot_df.xInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 80
plot_df['y'] = plot_df.yInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 70.5
plot_df = plot_df.sort_values(by='y')
plot_df = plot_df.pivot(index='y', columns='x', values='count')
plot_df = plot_df.reindex(columns=sorted(plot_df.columns))

fig=plt.figure(figsize=(10, 6.5))
ax=fig.add_subplot()
pcm = ax.imshow(plot_df, zorder=0, aspect="auto", extent=(-80,80, -70.5, 70.5), 
          cmap='Reds')
plt.colorbar(pcm)
generate_afl_oval('MCG')

In [ ]:
plot_df = score_probability.reset_index()
plot_df['x'] = plot_df.xInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 80
plot_df['y'] = plot_df.yInitialPoss_bin.apply(lambda s: np.round(s.values.left,3)).astype('float') * 70.5
plot_df = plot_df.sort_values(by='y')
plot_df = plot_df.pivot(index='y', columns='x', values='count')
plot_df = plot_df.reindex(columns=sorted(plot_df.columns))

fig=plt.figure(figsize=(10, 6.5))
ax=fig.add_subplot()
pcm = ax.imshow(plot_df, zorder=0, aspect="auto", extent=(-80,80, -70.5, 70.5), 
          cmap='Reds')
plt.colorbar(pcm)
generate_afl_oval('MCG')

## Transition matricies

In [ ]:
transitions = possession_summary.groupby(['xInitialPoss_bin','yInitialPoss_bin','xFinalPoss_bin', 'yFinalPoss_bin']).aggregate(shots = ('shotAtGoal', sum),
                                                                           goals = ('goals', sum),
                                                                           behinds = ('behind', sum),
                                                                           total_events = ('possessionNum', 'size'))

transitions.loc[x_bins[10],y_bins[10], :, :].sort_values(by='total_events',ascending=False).head(30)